<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/wisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-qqosb7qc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-qqosb7qc
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802823 sha256=60974cb407f313d9ca3195a281edb339d6a80c225a8f89f7c951c7c7025d1f94
  Stored in directory: /tmp/pip-ephem-wheel-cache-yanbghbp/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
import whisper
import json

In [ ]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 75.8MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
result = model.transcribe("./97388baa-c741-41eb-ac88-34fed51eca76.wav", verbose=True, fp16=False, language="ja") #ファイル指定

[00:00.000 --> 00:06.300] バック線がですが、ここまでフェースがシャットに上がってすごくいい感じなんですけど
[00:06.300 --> 00:14.720] 一旦この練習でですね、ここの位置の時に顔をヘッドの方に向けてみてください
[00:14.720 --> 00:24.960] 今この映像で見る限り、胸は少し映像から見る右下の方向に向いているのに対して
[00:24.960 --> 00:31.960] 腕は少し右肘が曲がって、手元の位置っていうのは胸の向きよりも少し
[00:31.960 --> 00:35.960] 自分の体に対して右に動いてしまっているんですね
[00:35.960 --> 00:42.960] なぜ顔をヘッドの方に向けて欲しいかというと、その向けてみると自分の体が回ってない
[00:42.960 --> 00:49.460] 胸と手元が一緒に同じ方向にないということが
[00:49.460 --> 00:53.460] 自分の体のセンターに手元がないということが、多分わかっていただけると思うので
[00:53.460 --> 00:59.960] カーフショットの練習でいいので、まずこのポジションの時に顔をヘッドの方に向けてみてください
[00:59.960 --> 01:06.960] それで胸と手元の向き、胸の中心に手元クリップがあって
[01:06.960 --> 01:11.460] 肘が、右肘がそれによって伸びると思いますので
[01:11.460 --> 01:18.960] ちょっとそのポジションを作って、右肘が伸びた状態のままダウンスイングをする練習をしてみてください
[01:18.960 --> 01:24.960] ここからトップに上がっていく時にさらに肘が曲がって横に引いていってしまって
[01:24.960 --> 01:29.960] 自分の体の胸の中心から外れた状態になってしまうので
[01:29.960 --> 01:34.960] そこをやっぱりちょっとこう直していきたいなという感じがまだします
